# 소셜 버즈 데이터 수집 및 분류
[목표]
- poi(관광지)별 2021년 날짜에 해당하는 소셜 버즈 데이터를 수집한 후, poi별 데이터셋 작성
[진행할 과정]

1. 네이버 블로그 크롤링
2. 크롤링 데이터 분류
[주의사항]
- 크롤링을 돌릴 때, social buzz 폴더안의 모든 데이터를 삭제하고 돌리셔야 합니다.


## 1차 : 네이버 블로그 크롤링
- 과정 1 : 2021년 모든 날짜를 리스트로 저장
- 과정 2 : poi리스트를 가져와 한 poi당 당 모든 요일 수인 365번 수행 (1.1~12.31)
- 과정 3 : 약 200개의 모든 poi들에 대해 과정 2 수행
[데이터셋]
- 관광지 : 제주도 내의 poi(관광지)의 명칭 저장.
[변수 설명]

1. 기본 변수
- poi : 각 poi(관광지 데이터)
- date_list : 2021-01-01 ~ 2021-12-31 사이의 모든 날짜들의 리스트
- search : 특정 관광지 지정 (크롤링 내의 반복문에서 활용)
2. url 변수
- url_first : startdate를 입력하기 전 url
- url_second : startdate를 입력한 이후 enddate를 입력하기 이전 url
- url_third : enddate 입력 이후, keyword, 즉 poi를 입력하기 이전 url
3. 크롤링 변수
- visit_list : poi명 (관광지명)을 리스트 형태로 나타냄 (1개의 관광지당 365번)
- count_list : 소셜 버즈량(1월1일부터 12월31일까지)
- count_num : 관광지 해당 요일의 소셜 버즈량을 int형태로 나타낸 값
- result : 관광지, 소셜 버즈량, 날짜 데이터를 모두 가져와 합친 데이터프레임
[크롤링 과정]

- 네이버 블로그 사이트에서, 각 관광지 별 2021년 1월 1일부터 2021년 12월 31일까지 일마다 몇개의 블로그 개수가 있는지 크롤링 후 데이터프레임으로 저장 -> 소셜버즈량의 데이터로 사용

- 작동 원리 : 하나의 관광지별로 1월1일부터 12월31일까지의 일별 개수를 크롤링한 다음, 다음 관광지로 넘기기

[결과 데이터셋]

- 관광지명 : 1월1일부터 12월31일까지의 개수인 365개의 데이터가 각 관광지별로 존재( ex. 성산일출봉 365개)
- 날짜 : 각 요일, 2021-01-01부터 2021-12-31까지 각 관광지 별 365개 존재
- 소셜버즈량 수 : 각 관광지별, 일별로 몇건의 블로그 개수가 있는지 결과

In [1]:
# 라이브러리 불러오기
from bs4 import BeautifulSoup
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
from selenium.webdriver.common.action_chains import ActionChains

import warnings
warnings.filterwarnings('ignore')

In [9]:
# 2021-01-01에서 2021-12-31 사이 모든 날짜를 크롤링하는 함수 만들어서 가져오기
from datetime import datetime, timedelta
# 해당 범위 내의 날짜들을 리스트로 지정해주는 함수
def date_range(start, end):
    start = datetime.strptime(start, "%Y-%m-%d")
    end = datetime.strptime(end, "%Y-%m-%d")
    dates = [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end-start).days+1)]
    return dates
# 2021-01-01 ~ 2021-12-31 사이의 날짜를 리스트로 지정
date_list = date_range("2022-01-01", "2022-12-31")

In [4]:
# date_list는 str형식이기 때문에 크롤링 과정에서 굳이 바꿔줄 필요 없음
date_list[:5]

['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04', '2022-01-05']

### 디버깅코드

In [2]:
df = pd.read_csv('충남관광지리스트.csv')
df.head()

,관광지명,광역시/도,시/군/구,도로명주소,중분류 카테고리,소분류 카테고리,검색건수
0,아산스파비스,충청남도,아산시,충남 아산시 아산온천로157번길 67-0,체험관광,웰니스관광,49046
1,파라다이스스파도고,충청남도,아산시,충남 아산시 도고온천로 176-1,체험관광,웰니스관광,35164
2,파라다이스스파도고,충청남도,아산시,충남 아산시 도고온천로 176-0,체험관광,웰니스관광,33766
3,하늘물빛정원,충청남도,금산군,충남 금산군 검한1길 156-0,체험관광,농/산/어촌체험,22495
4,팜카밀레허브농원,충청남도,태안군,충남 태안군 우운길 56-19,체험관광,농/산/어촌체험,21646


# 2022년 1년동안 매일마다의 네이버 블로그 소셜 버즈량 크롤링

In [7]:
url = 'https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=PERIOD&orderBy=sim&startDate=2022-01-01&endDate=2022-12-31&keyword=성산일출봉'
driver = webdriver.Chrome(executable_path = 'C:/Users/chromedriver.exe') # 드라이버 열기

# 정보를 담을 리스트
visit_list = [] # 관광지 리스트
count_list = [] # 소셜버즈량 리스트

for i in range(len(df)):
    url_first = 'https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=PERIOD&orderBy=sim&startDate=2022-01-01&endDate=2022-12-31&keyword='
    search = df['관광지명'][i]


    url = url_first + search
    driver.get(url)
    time.sleep(1.5)

    try : 
        count = driver.find_element(By.XPATH, '//*[@id="content"]/section/div[1]/div[2]/span/span/em')
    except : # 잘 못찾는 경우가 있어서 except에도 같은 경우를 넣어줌. 이렇게 되면 못찾는 일이 없음
        count = driver.find_element(By.XPATH, '//*[@id="content"]/section/div[1]/div[2]/span/span/em')
    finally : 
        # 몇건이 조회되었는지 가져옴
        count_num = count.text # 텍스트만 추출
        count_num = count_num.replace(',','') # ,을 빈칸으로 변환
        count_num = int(count_num.split('건')[0]) # 앞의 숫자만 가져오고 형태 변환
        count_list.append(count_num) # count_list에 추가
        visit_list.append(search) # visit_list에 일자 수만큼 관광지명 넣기
        print(search) # 하나의 관광지에 해당 요일에 대한 크롤링이 완료되었을 때 출력

/var/folders/gx/q7vn3yv17rvb0vy6chbbbztw0000gn/T/ipykernel_779/3412049110.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = 'C:/Users/chromedriver.exe') # 드라이버 열기


아산스파비스
파라다이스스파도고
파라다이스스파도고
하늘물빛정원
팜카밀레허브농원
아그로랜드태신목장
아름다운정원화수목
아산온천
덕산싸이판온천
선도리갯벌체험마을
스플라스워터파크
신천탕
진산리갯벌체험장
풍세관광농원
한밭관광농원
금강온천
병술만어촌체험마을
아이러브팜핑
다울관광농원
남면관광농원
딸기향농촌테마공원
한채당한옥체험관
세재골농원
해오름관광농원
아일랜드리솜오아식스선셋스파
내생애첫딸기
맘맘스
중리어촌체험마을
계룡산온천
해바라기올래정원
만대어촌체험마을
숲속애빛의정원
히읗히읗
중장리농촌체험휴양마을
덕바위마을
양촌성구랜드
덕화온천장
수림휴원
일월관광농원
세재골농원
토끼별미술관
은행마을
장승배기체험농장
팔봉갯벌체험장
황도바지락유어체험장
서해관광랜드
보령팜
허브랜드
부여기와마을
덕암스파
놀뫼종합체험학습장
서천송림갯벌체험장
웅도어촌체험마을
공주치즈스쿨
상양관광농원
홍산관광농원
세심천온천
조팝꽃피는마을희망센터
월하성어촌체험마을
피구지관광농원
이시돌쉼터
아리랜드
갈숲마을
덕산온천족욕장
다울관광농원
산촌생태마을
매화둠벙마을체험관
오색꽃차충의치유농원
하늘빛정원
홍성온천
개삼터관광공원
마곡온천
보곡산골마을
효덕목장
효자사슴농장관광농원
강당골산촌체험관
공방고운
라벤더공방
가경주체험마을
천안상록리조트아쿠아피아
아바바
친환경까치마을농촌체험관
왕산어촌체험휴양마을
노을지는갯마을
빙도녹색농촌체험마을
바느질공방K
연주농원
그린농원
농업회사법인자연과함께힐링
서경숙알밤농장
벧엘딸기농장
신진도갯벌체험장
솔뫼석주원
퍼퓨머일랑
서산관광농원
열두바구니
딸기야농원
이안에식물
아가다리랜드
도고온천
금송딸기농원
꿈꾸는달님도자기핸드페인팅공방
온직다원
효자딸기농원
나오리생태예술원
칠갑산산꽃마을
갈두천권역커뮤니티센터
해피팜스토리
물여울마을
검은들딸기농장
승일목장미호치즈
꾀꼴성마을
왕대골농촌체험마을
서산관광농원
청신목장
석송목장
테라도자기공방
백석올미농촌체험휴양마을
늘샘농원
송곡리은행나무길
대천해수욕장스카이바이크
롯데스카이힐부여컨트리클럽
안면도국제꽃박람회
당진도비도항
예당호모노레일
경륜경정천안지점
군헌갯벌체험장
충청북도해

MaxRetryError: HTTPConnectionPool(host='localhost', port=49352): Max retries exceeded with url: /session/b65eee6f155e57d7f92fb44e5f63cd38/element/C6F3A1443D6347953D493BBDEB965019_element_44568/text (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fed7a0598b0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [11]:
# 872~
driver = webdriver.Chrome(executable_path = 'C:/Users/chromedriver.exe') # 드라이버 열기
for i in range(872, len(df)):
    url_first = 'https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=PERIOD&orderBy=sim&startDate=2022-01-01&endDate=2022-12-31&keyword='
    search = df['관광지명'][i]


    url = url_first + search
    driver.get(url)
    time.sleep(1.5)

    try : 
        count = driver.find_element(By.XPATH, '//*[@id="content"]/section/div[1]/div[2]/span/span/em')
    except : # 잘 못찾는 경우가 있어서 except에도 같은 경우를 넣어줌. 이렇게 되면 못찾는 일이 없음
        count = driver.find_element(By.XPATH, '//*[@id="content"]/section/div[1]/div[2]/span/span/em')
    finally : 
        # 몇건이 조회되었는지 가져옴
        count_num = count.text # 텍스트만 추출
        count_num = count_num.replace(',','') # ,을 빈칸으로 변환
        count_num = int(count_num.split('건')[0]) # 앞의 숫자만 가져오고 형태 변환
        count_list.append(count_num) # count_list에 추가
        visit_list.append(search) # visit_list에 일자 수만큼 관광지명 넣기
        print(search) # 하나의 관광지에 해당 요일에 대한 크롤링이 완료되었을 때 출력

/var/folders/gx/q7vn3yv17rvb0vy6chbbbztw0000gn/T/ipykernel_779/264881677.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = 'C:/Users/chromedriver.exe') # 드라이버 열기


원풍식당
백화산가든[한식]
대천키조개삼합
대안정육점식당
진국집
보경이네
온천손칼국수
청운식당[중식]
비브레이브천안점
통삼국본점
망향비빔국수아산점
신준호카페
산내들
홍북식당
예가
서산회관
배가짬뽕[중식]
미성삼색장어
삽교호관광지
독립기념관
상화원
청산수목원
안면도쥬라기박물관
천안축구센터
삽교호놀이동산
태안세계튤립축제
백제문화단지
유구색동수국정원
장항스카이워크
국립부여박물관
개화예술공원
중앙호수공원
국립공주박물관
롯데시네마천안불당
피나클랜드
CGV천안펜타포트
천안예술의전당
천안박물관
아미미술관
계룡세계군문화엑스포
삽교호함상공원
태안빛축제
선샤인랜드
CGV천안시청
세계꽃식물원
홍성남당항대하축제
천안상록리조트
금강신관공원
장고항실치축제
모나무르
아산생태곤충원
메가박스천안
CGV당진
코리아플라워파크
부여서동연꽃축제
사랑나무
석장리박물관
모나무르
신정호유원지
보령석탄박물관
천안삼거리공원
보령머드축제
메가박스홍성내포
CGV천안터미널
롯데시네마아산터미널
금강하구둑관광단지
예산장터삼국축제
롯데시네마서산
장영실과학관
금산인삼축제
홍성남당항새조개축제
명품칠갑산얼음분수축제
온양민속박물관
죽도관광지
천안홍대용과학관
태안꽃박람회
소노벨천안오션어드벤처
CGV서산
CGV천안
2022보령해양머드박람회
연산문화창고
탑정호수변생태공원
연미산자연미술공원
신방공원
한산모시관
당진전력문화홍보관
백제문화제공주
청수호수공원
성정공원
롯데시네마당진
유관순열사기념관
너티차일드천안점
도솔광장
공주시립박찬호야구장
보령에너지월드
만리포해수욕장전망대
점핑고신불당점
오션어드벤처
천안생활체육공원
안면도백사장대하축제
선장축구장
속동전망대
예산황새공원
서동요오픈세트장
계룡산자연사박물관
메가박스공주
천안타운홀전망대
구드래조각공원
고운식물원
보령해저터널홍보관
한화이글스서산경기장
윤봉길의사기념관
현대자동차문화관
봉서홀
둔포체육공원
보령문화의전당
삽교호바다공원
천안자동차극장
미스터션샤인촬영지(선샤인랜드)
천안시민체육공원
백제군사박물관
칠지공원
선샤인스튜디오
서산국화축제
예산시네마
상록리조트상록랜드
당림미술관
CGV홍성
고대지구

In [12]:
result = pd.DataFrame()
result['관광지'] = visit_list
result['언급량'] = count_list

In [16]:
result.head()

,관광지,언급량
0,아산스파비스,1249
1,파라다이스스파도고,1055
2,파라다이스스파도고,1055
3,하늘물빛정원,738
4,팜카밀레허브농원,515


In [17]:
result.to_csv('관광지네이버블로그언급량.csv',index=False, encoding='utf-8-sig')

In [8]:
df[df['관광지명']=='해어름카페']

,관광지명,광역시/도,시/군/구,도로명주소,중분류 카테고리,소분류 카테고리,검색건수
643,해어름카페,충청남도,당진시,충남 당진시 매산해변길 144-0,음식,카페/찻집,34104
871,해어름카페,충청남도,당진시,충남 당진시 매산해변길 144-0,음식,전문음식,4880


In [14]:
len(result)

1636

In [15]:
len(df)

1636